In [1]:
import numpy as np
import tensorflow as tf
import os
import json

# Tagging Using the BM Dataset
Using the BM dataset tags, we use a pretrained InceptionNet train a classifer to tag images

In [2]:

ROOT_DIR = '/data/images_raw_1M/'
MODEL_ROOT = '/tmp/'
#ROOT_DIR = '/data/captioning/img_100_gs/'
#ROOT_DIR = '/Users/luda/Projects/SherlockNet/img_100_gs/'
#MODEL_ROOT = '/Users/luda/Projects/SherlockNet/tmp/boat/'


In [6]:

def create_graph(modelFullPath):
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')


def run_inference_on_images(imageFns, c, tag):
    print "category: ", c
    print "tag: ", tag
    res = []
#     if not tf.gfile.Exists(imagePath):
#         tf.logging.fatal('File does not exist %s', imagePath)
#         return answer
    modelFullPath = MODEL_ROOT + tag + '/output_graph.pb'
    labelsFullPath = MODEL_ROOT + tag + '/output_labels.txt'

    f = open(labelsFullPath, 'rb')
    lines = f.readlines()
    labels = [str(w).replace("\n", "") for w in lines]
    # Creates graph from saved GraphDef.
    create_graph(modelFullPath)
    res.append((len(imageFns), labels[0], labels[1]))

    with tf.Session() as sess:
        count = 0
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        for imageFn in imageFns:
            count +=1
            image_data = tf.gfile.FastGFile(ROOT_DIR + c + '/'  + imageFn, 'rb').read()
            predictions = sess.run(softmax_tensor,
                                   {'DecodeJpeg/contents:0': image_data})
            predictions = np.squeeze(predictions)

            top_k = predictions.argsort()[-2:][::-1]  # Getting 2 scores
            res.append((imageFn, predictions[top_k[0]], predictions[top_k[1]]))

            print res
            for node_id in top_k:
                human_string = labels[node_id]
                score = predictions[node_id]
                print('%s (score = %.5f)' % (human_string, score))
            if (count % 100 == 0):
                print count

    return res, labels

In [7]:
tf.reset_default_graph()
tags = ['boat', 'mountain', 'river', 'bridge']
categories = ['landscapes', 'nature']
#tags = ['boat']
for c in categories:
    for tag in tags:
        res, labels = run_inference_on_images(os.listdir(ROOT_DIR + c), c, tag)
        with open('/data/captioning/bm_tagging_' + c + '_' + tag + '.json', 'w') as outfile:
            json.dump(res, outfile)

category:  landscapes
tag:  boat
[(118816, 'not boat', 'boat'), ('000579900_0_000115_1_.jpg', 0.50186419, 0.49813583)]
not boat (score = 0.50186)
boat (score = 0.49814)
[(118816, 'not boat', 'boat'), ('000579900_0_000115_1_.jpg', 0.50186419, 0.49813583), ('003914899_02_000606_1_.jpg', 0.78092724, 0.21907276)]
boat (score = 0.78093)
not boat (score = 0.21907)
[(118816, 'not boat', 'boat'), ('000579900_0_000115_1_.jpg', 0.50186419, 0.49813583), ('003914899_02_000606_1_.jpg', 0.78092724, 0.21907276), ('003885107_0_000065_1_.jpg', 0.66353935, 0.33646065)]
not boat (score = 0.66354)
boat (score = 0.33646)
[(118816, 'not boat', 'boat'), ('000579900_0_000115_1_.jpg', 0.50186419, 0.49813583), ('003914899_02_000606_1_.jpg', 0.78092724, 0.21907276), ('003885107_0_000065_1_.jpg', 0.66353935, 0.33646065), ('000735816_0_000172_1_.jpg', 0.73402327, 0.26597673)]
not boat (score = 0.73402)
boat (score = 0.26598)
[(118816, 'not boat', 'boat'), ('000579900_0_000115_1_.jpg', 0.50186419, 0.49813583), ('00

KeyboardInterrupt: 

In [8]:

c = 'test'
tag = 'test'
res = ['test']
with open('/data/captioning/bm_tagging_' + c + '_' + tag + '.json', 'w') as outfile:
    json.dump(res, outfile)